## Phase 1: Setup, Data Loading, and Parameter Definition. 
This code will import our libraries, load the smart_grid_dataset_2024.csv file, and set up all the "Given" parameters from our Energy_qubo.pdf document.

In [20]:
# Step 1: Setup, Data Loading, and Parameter Definition
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("Phase 1: Setup & Data Preparation")

#  1. Load Data 
try:
    df = pd.read_csv("smart_grid_dataset_2024.csv")
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
except FileNotFoundError:
    print("ERROR: 'smart_grid_dataset_2024.csv' not found.")
    # In a real notebook, we'd stop execution here

#  2. Create Time-Series Arrays (The "Givens") 
# Filter for our 24-hour period (Jan 1, 2024)
df_24h = df[df['Timestamp'].dt.date == pd.to_datetime('2024-01-01').date()].copy()

if len(df_24h) != 96:
    print(f"Warning: Expected 96 time steps, but found {len(df_24h)}.")

# Total time steps (T = 96)
T = len(df_24h) 

# B_t: Baseline Load (kW)
B_t = df_24h['Power Consumption (kW)'].values

# C_t: Energy Price (USD/kWh)
C_t = df_24h['Electricity Price (USD/kWh)'].values

# R_t: Renewable Production (kW)
R_t = (df_24h['Solar Power (kW)'] + df_24h['Wind Power (kW)']).values

print(f"Data loaded for {T} time steps.")

# 3. Define Appliance Parameters (The "Problem Space") 
# As defined in Energy_qubo.pdf
# Note: We convert time windows to integer indices (0-95)
# 8:00 AM = 8*4 = 32
# 8:00 PM = 20*4 = 80
# 10:00 PM = 22*4 = 88
'''appliances = {
    'ev_charger': {
        'P': 9.0,  # kW
        'L': 16,   # 16 steps = 4 hours
        # Window: 10:00 PM (88) to 6:00 AM (ends at step 23)
        'window': list(range(88, 96)) + list(range(0, 24 - 16 + 1)) 
    },
    'washing_machine': {
        'P': 1.5,  # kW
        'L': 4,    # 4 steps = 1 hour
        # Window: 8:00 AM (32) to 10:00 PM (ends at step 87)
        'window': list(range(32, 88 - 4 + 1))
    },
    'dishwasher': {
        'P': 1.8,  # kW
        'L': 6,    # 6 steps = 1.5 hours
        # Window: 8:00 PM (80) to 5:00 AM (ends at step 19)
        'window': list(range(80, 96)) + list(range(0, 20 - 6 + 1))
    }
}'''

# Note on windows: The window list contains all valid START times.
# e.g., washing machine must END by step 87, so it must START by 87 - 4 = 83.
# We've adjusted the range() function accordingly.



# --- 3. Define Appliance Parameters (PRUNED) ---
# We are adding a 'step=4' to the window ranges to reduce the
# number of qubits. This restricts starts to hourly intervals.

print("Redefining Appliance Parameters (Pruned for Simulation)")

appliances = {
    'ev_charger': {
        'P': 9.0,  # kW
        'L': 16,   # 16 steps = 4 hours
        # Window: 10:00 PM (88) to 6:00 AM (ends at step 23)
        'window': list(range(88, 96, 4)) + list(range(0, 24 - 16 + 1, 4)) 
    },
    'washing_machine': {
        'P': 1.5,  # kW
        'L': 4,    # 4 steps = 1 hour
        # Window: 8:00 AM (32) to 10:00 PM (ends at step 87)
        'window': list(range(32, 88 - 4 + 1, 4))
    },
    'dishwasher': {
        'P': 1.8,  # kW
        'L': 6,    # 6 steps = 1.5 hours
        # Window: 8:00 PM (80) to 5:00 AM (ends at step 19)
        'window': list(range(80, 96, 4)) + list(range(0, 20 - 6 + 1, 4))
    }
}

print("Appliance parameters (pruned) are now defined.")
#print("Appliance parameters defined.")
print("Phase 1 Complete ")

Phase 1: Setup & Data Preparation
Data loaded for 96 time steps.
Redefining Appliance Parameters (Pruned for Simulation)
Appliance parameters (pruned) are now defined.
Phase 1 Complete 


- We will now write the code to translate our QUBO formulation from Energy_qubo.pdf into a format PennyLane understands (an Ising Hamiltonian). 
- This involves building the $h$ (single-qubit) and $J$ (two-qubit) coefficients.
- Step 2: Building the Ising HamiltonianThis code block defines our decision variables (qubits) and then meticulously builds the $h$ and $J$ coefficients for each of our three Hamiltonian terms, based on the equations in our PDF

In [21]:
print("\n Phase 2: Building the Ising Hamiltonian ")

#  4. Define Qubit Mapping (Decision Variables) 
# We need to map each possible start time for each appliance to a unique qubit.
# x_{i,t} -> qubit_index

qubit_map = {} # {('appliance_name', start_time): qubit_index}
reverse_map = {} # {qubit_index: ('appliance_name', start_time)}
current_qubit_index = 0

for app_name, details in appliances.items():
    for start_time in details['window']:
        qubit_map[(app_name, start_time)] = current_qubit_index
        reverse_map[current_qubit_index] = (app_name, start_time)
        current_qubit_index += 1

n_qubits = len(qubit_map)
print(f"Total number of decision variables (qubits): {n_qubits}")

# --- 5. Define Hyperparameters (A, B, C) ---
# These are the "lambda" knobs we discussed.
A = 1.0  # Weight for Cost
B = 1.5  # Weight for Peak Load (prioritizing this, as discussed)
C = 50.0 # Penalty for constraints (must be the largest)
print(f"Hyperparameters: A={A}, B={B}, C={C}")

# --- 6. Build Hamiltonian Coefficients (h, J) ---
# We will build the coefficients for the Ising model:
# H = sum_i(h_i * s_i) + sum_{i<j}(J_{ij} * s_i * s_j) + Offset
# We use the substitution x = (1 - s) / 2
# A linear term x_i becomes: (1 - s_i) / 2 = 0.5 - 0.5*s_i
# A quadratic term x_i*x_j becomes: (1 - s_i)(1 - s_j) / 4 = 0.25 - 0.25*s_i - 0.25*s_j + 0.25*s_i*s_j

h = {}  # h_i coefficients (single-qubit terms)
J = {}  # J_ij coefficients (two-qubit interaction terms)
offset = 0.0 # Constant offset

# 6a. H_cost_term 
# H_cost = A * sum_{i,t} (C_{i,t} * x_{i,t})
#
print("Building H_cost...")
for (app_name, start_time), qubit_idx in qubit_map.items():
    app = appliances[app_name]
    P = app['P']
    L = app['L']
    
    # Pre-calculate the cost C_{i,t} for this start time
    cost_of_run = 0.0
    for t in range(start_time, start_time + L):
        # Use modulo T to handle windows wrapping past midnight
        cost_of_run += C_t[t % T] 
    
    C_i_t = P * cost_of_run
    
    # H_cost = A * C_i_t * x_i
    # Substitute x_i = (1 - s_i) / 2
    # H_cost = A * C_i_t * (0.5 - 0.5 * s_i)
    # H_cost = (0.5 * A * C_i_t) - (0.5 * A * C_i_t * s_i)
    
    offset += A * C_i_t * 0.5
    h[qubit_idx] = h.get(qubit_idx, 0.0) - (A * C_i_t * 0.5)

# 6b. H_constraint_term 
# H_constraint = C * sum_i ( (sum_t(x_{i,t}) - 1)^2 )
# H_constraint = C * sum_i ( (sum_t(x_{i,t})^2) - 2*sum_t(x_{i,t}) + 1 )
# Since x_i^2 = x_i for binary:
# H_constraint = C * sum_i ( sum_t(x_{i,t}) + 2*sum_{t<t'}(x_{i,t}*x_{i,t'}) - 2*sum_t(x_{i,t}) + 1 )
# H_constraint = C * sum_i ( 2*sum_{t<t'}(x_{i,t}*x_{i,t'}) - sum_t(x_{i,t}) + 1 )
#
print("Building H_constraint...")
for app_name, app_details in appliances.items():
    app_qubits = [qubit_map[(app_name, t)] for t in app_details['window']]
    
    # Add constant offset term: C * sum_i(1)
    offset += C
    
    # Add linear term: -C * sum_i(sum_t(x_{i,t}))
    for qubit_idx in app_qubits:
        # Term is -C * x_i
        # Substitute x_i = (0.5 - 0.5 * s_i)
        # Term = -C * (0.5 - 0.5 * s_i) = (-0.5 * C) + (0.5 * C * s_i)
        offset -= 0.5 * C
        h[qubit_idx] = h.get(qubit_idx, 0.0) + (0.5 * C)
        
    # Add quadratic term: 2*C * sum_i(sum_{t<t'}(x_{i,t}*x_{i,t'}))
    for i in range(len(app_qubits)):
        for j in range(i + 1, len(app_qubits)):
            q_i = app_qubits[i]
            q_j = app_qubits[j]
            
            # Term is 2 * C * x_i * x_j
            # Substitute x_i*x_j = 0.25 - 0.25*s_i - 0.25*s_j + 0.25*s_i*s_j
            # Term = 2*C * (0.25 - 0.25*s_i - 0.25*s_j + 0.25*s_i*s_j)
            # Term = (0.5*C) - (0.5*C*s_i) - (0.5*C*s_j) + (0.5*C*s_i*s_j)
            
            offset += 0.5 * C
            h[q_i] = h.get(q_i, 0.0) - (0.5 * C)
            h[q_j] = h.get(q_j, 0.0) - (0.5 * C)
            
            # Ensure J key is (min_idx, max_idx)
            J_key = tuple(sorted((q_i, q_j)))
            J[J_key] = J.get(J_key, 0.0) + (0.5 * C)


# 6c. H_peak_term_v2 
# This is the most complex term.
# H_peak = B * sum_t ( (B_t + L_deferrable(t) - R_t)^2 )
# H_peak = B * sum_t ( (L_deferrable(t) + (B_t - R_t))^2 )
# H_peak = B * sum_t ( L_deferrable(t)^2 + 2*L_deferrable(t)*(B_t - R_t) + (B_t - R_t)^2 )
#
print("Building H_peak...")
for t in range(T):
    # Constant term: B * (B_t - R_t)^2
    net_baseline = B_t[t] - R_t[t]
    offset += B * (net_baseline ** 2)
    
    # We need to find all x_{i,k} that are 'ON' at time t
    # L_deferrable(t) = sum_i (P_i * sum_{k=t-L_i+1 to t} (x_{i,k}))
    #
    
    L_def_terms = [] # List of (qubit_index, power_P_i)
    for (app_name, start_time), qubit_idx in qubit_map.items():
        app = appliances[app_name]
        P = app['P']
        L = app['L']
        
        # Check if an appliance starting at 'start_time' is ON at time 't'
        # This handles wrapping around midnight
        is_on = False
        if start_time + L > T: # Wraps around
            if (t >= start_time) or (t < (start_time + L) % T):
                is_on = True
        else: # Does not wrap
            if (t >= start_time) and (t < start_time + L):
                is_on = True
        
        if is_on:
            L_def_terms.append((qubit_idx, P))

    # Linear term: B * 2 * L_deferrable(t) * (B_t - R_t)
    # Term = 2 * B * net_baseline * sum(P_i * x_i)
    for qubit_idx, P_i in L_def_terms:
        # Term = (2 * B * net_baseline * P_i) * x_i
        # Substitute x_i = (0.5 - 0.5 * s_i)
        # Term = (B * net_baseline * P_i) - (B * net_baseline * P_i * s_i)
        
        coeff = B * net_baseline * P_i
        offset += coeff
        h[qubit_idx] = h.get(qubit_idx, 0.0) - coeff

    # Quadratic term: B * (L_deferrable(t)^2)
    # Term = B * (sum(P_i * x_i))^2
    # Term = B * (sum_i( (P_i*x_i)^2 ) + 2*sum_{i<j}( P_i*P_j*x_i*x_j ))
    # Since x_i^2 = x_i (binary):
    # Term = B * (sum_i( P_i^2 * x_i ) + 2*sum_{i<j}( P_i*P_j*x_i*x_j ))
    
    # Quadratic's linear part: B * sum_i( P_i^2 * x_i )
    for qubit_idx, P_i in L_def_terms:
        # Term = B * P_i^2 * x_i
        # Substitute x_i = (0.5 - 0.5 * s_i)
        # Term = (0.5 * B * P_i^2) - (0.5 * B * P_i^2 * s_i)
        
        coeff = 0.5 * B * (P_i**2)
        offset += coeff
        h[qubit_idx] = h.get(qubit_idx, 0.0) - coeff

    # Quadratic's quadratic part: B * 2 * sum_{i<j}( P_i*P_j*x_i*x_j )
    for i in range(len(L_def_terms)):
        for j in range(i + 1, len(L_def_terms)):
            q_i, P_i = L_def_terms[i]
            q_j, P_j = L_def_terms[j]
            
            # Term = 2 * B * P_i * P_j * x_i * x_j
            # Substitute x_i*x_j = 0.25 - 0.25*s_i - 0.25*s_j + 0.25*s_i*s_j
            # Term = (0.5*B*P_i*P_j) - (0.5*B*P_i*P_j*s_i) - (0.5*B*P_i*P_j*s_j) + (0.5*B*P_i*P_j*s_i*s_j)
            
            coeff = 0.5 * B * P_i * P_j
            
            offset += coeff
            h[q_i] = h.get(q_i, 0.0) - coeff
            h[q_j] = h.get(q_j, 0.0) - coeff
            
            J_key = tuple(sorted((q_i, q_j)))
            J[J_key] = J.get(J_key, 0.0) + coeff

# 7. Create PennyLane Hamiltonian Object 
h_coeffs = list(h.values())
h_obs = [qml.PauliZ(i) for i in h.keys()]

J_coeffs = list(J.values())
J_obs = [qml.PauliZ(i) @ qml.PauliZ(j) for i, j in J.keys()]

H_Ising = qml.Hamiltonian(h_coeffs + J_coeffs, h_obs + J_obs)

print(f"Built Ising Hamiltonian with {len(h_coeffs)} single-qubit terms")
print(f"and {len(J_coeffs)} two-qubit terms.")
print(f"Constant Offset (ignored by optimizer): {offset}")
print(" Phase 2 Complete ")


 Phase 2: Building the Ising Hamiltonian 
Total number of decision variables (qubits): 27
Hyperparameters: A=1.0, B=1.5, C=50.0
Building H_cost...
Building H_constraint...
Building H_peak...
Built Ising Hamiltonian with 27 single-qubit terms
and 154 two-qubit terms.
Constant Offset (ignored by optimizer): 182455.3639871662
 Phase 2 Complete 


### Summary of What We've Done
- Qubit Mapping: We created a qubit_map that assigns a unique qubit index (e.g., 0 to n_qubits-1) to every valid start-time decision variable. We found we have 86 qubits for this problem. 
- Hyperparameters: We set our weights A, B, and C.
- Substitution: We systematically applied the $x = (1 - s) / 2$ substitution to all three parts of our Hamiltonian from Energy_qubo.pdf.
- Coefficient Building: We iterated through all appliances and time steps to build the h (single-qubit) and J (two-qubit) coefficient dictionaries. This was complex, especially for the constraint and peak-load terms, but it's the core of the problem.
- Hamiltonian Object: We combined these coefficients into a qml.Hamiltonian object. This H_Ising is the final problem object that PennyLane will solve.

## Phase 3. 
- We will define our high-performance simulator (lightning.qubit) and then create the QAOA "cost function" (the QNode) that PennyLane will learn to optimize. This QNode takes our H_Ising Hamiltonian from Step 2 as its objective.

Step 3: Define the QAOA Circuit

In [22]:
print("\n Phase 3: Define the QAOA Circuit ")

# 8. Select Device
# We use "lightning.qubit" for a high-performance C++ simulator
try:
    dev = qml.device("lightning.qubit", wires=n_qubits)
    print(f"Device selected: lightning.qubit with {n_qubits} wires.")
except qml.DeviceError:
    print("Warning: 'pennylane-lightning' not installed. Falling back to 'default.qubit'.")
    print("For faster performance, run: pip install pennylane-lightning")
    dev = qml.device("default.qubit", wires=n_qubits)

# 9. Define QAOA Hyperparameters 
p = 2  # Number of QAOA layers (p)
print(f"QAOA layers (p) = {p}")

# --- 10. Define the Mixer Hamiltonian ---
# This is the standard QAOA mixer, a sum of PauliX on all qubits
mixer_H = qml.Hamiltonian(
    [1.0] * n_qubits,
    [qml.PauliX(i) for i in range(n_qubits)]
)
print("Mixer Hamiltonian defined.")

# 11. Define the Cost Function (QNode) 
# This circuit manually builds the QAOA layers.
# This is the modern, robust way and replaces the old 'QAOAAnsatz'.

@qml.qnode(dev)
def cost_circuit(params):
    # 1. Start in the |+> state (superposition)
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    
    # 2. Apply p layers of Cost and Mixer Hamiltonians
    for i in range(p):
        gamma = params[i, 0]  # Cost parameter
        beta = params[i, 1]   # Mixer parameter
        
        # Apply cost Hamiltonian: exp(-i * gamma * H_Ising)
        # We use ApproxTimeEvolution which is a standard template
        qml.templates.ApproxTimeEvolution(H_Ising, gamma, 1)
        
        # Apply mixer Hamiltonian: exp(-i * beta * mixer_H)
        qml.templates.ApproxTimeEvolution(mixer_H, beta, 1)
    
    # 3. Measure the expected value of our cost Hamiltonian
    return qml.expval(H_Ising)

# 12. Initialize Parameters 
# We need p layers, and each layer has 2 parameters (gamma, beta)
np.random.seed(42) # for reproducibility
params = np.random.rand(p, 2) * 0.1 # Shape (p, 2)

print("Cost circuit and initial parameters are defined.")
print(f"Parameter shape (p, 2): {params.shape}")
print("Phase 3 Complete")


 Phase 3: Define the QAOA Circuit 
Device selected: lightning.qubit with 27 wires.
QAOA layers (p) = 2
Mixer Hamiltonian defined.
Cost circuit and initial parameters are defined.
Parameter shape (p, 2): (2, 2)
Phase 3 Complete


### Summary of What We've Done
- Device: We've instantiated the lightning.qubit simulator, which will run our QAOA circuit very quickly.

- QAOA Layers: We've set p=2, meaning our circuit will have two layers of the cost Hamiltonian and two layers of the mixer Hamiltonian.

- Cost QNode: We've created cost_circuit, a quantum function that PennyLane can differentiate and optimize. It uses the qml. QAOAAnsatz template, which saves us from having to write the circuit manually.

- Parameters: We've created an initial set of random angles (params) to give the optimizer a starting point.

In [23]:
# Save Hamiltonian coefficients and operators to a text file
filename = "ising_hamiltonian.txt"

try:
    with open(filename, "w") as f:
        f.write("Single-qubit terms (h):\n")
        for coeff, obs in zip(h_coeffs, h_obs):
            f.write(f"{coeff}: {obs}\n")
        f.write("\nTwo-qubit terms (J):\n")
        for coeff, obs in zip(J_coeffs, J_obs):
            f.write(f"{coeff}: {obs}\n")
        f.write(f"\nConstant Offset: {offset}\n")
    print(f"Successfully saved Hamiltonian to {filename}")
except Exception as e:
    print(f"Error saving Hamiltonian: {e}")

Successfully saved Hamiltonian to ising_hamiltonian.txt


## Phase 4. 
- This block will create an optimizer, set up the training loop, and run it to find the best parameters for our QAOA circuit.

In [ ]:
import time

print("\n Phase 4: Run the Optimization ")

# 12. Set Optimizer and Training Hyperparameters 
opt = qml.AdamOptimizer(stepsize=0.01)
steps = 300  # We'll run 300 steps. This may take 1-5 minutes.
print(f"Optimizer: Adam, Steps: {steps}")

# We'll store our optimized parameters here
optimized_params = params

# We'll store the cost history to plot later
cost_history = []

# 13. The Training Loop
print("Starting optimization... (This may take a few minutes)")
start_time = time.time()

for i in range(steps):
    # This is the core line:
    # 1. It calls cost_circuit(optimized_params)
    # 2. It calculates the gradient (autodiff)
    # 3. The optimizer uses the gradient to update the params
    optimized_params = opt.step(cost_circuit, optimized_params)
    
    # We'll record the cost at each step
    cost = cost_circuit(optimized_params)
    cost_history.append(cost)
    
    # Print progress every 20 steps
    if (i + 1) % 20 == 0:
        print(f"Step {i+1:5d} | Cost = {cost:12.6f}")

end_time = time.time()
print("Optimization Complete")
print(f"Total time: {end_time - start_time:.2f} seconds")
print(f"Final cost (minimized energy): {cost_history[-1]:.6f}")

# 14. Plot Cost History 
plt.figure(figsize=(10, 4))
plt.plot(cost_history)
plt.title("QAOA Cost History")
plt.xlabel("Optimization Steps")
plt.ylabel("Cost (Expected Energy)")
plt.grid(True)
plt.savefig("qaoa_cost_history.png")
print("Saved cost history plot to 'qaoa_cost_history.png'")


 Phase 4: Run the Optimization 
Optimizer: Adam, Steps: 300
Starting optimization... (This may take a few minutes)
